In [1]:
# A horribly convoluted code for creating the LaTex tables. If you are any good at 
# coding (Steven), I know it could have been done a lot better... but it works (I think).

from astropy.table import Table
from tabulate import tabulate

source_dir = '../../fits/'
data_file = 'volume_limited_sample.fits'
debiased_file = 'debiased_volume_limited_sample.fits'

In [2]:
def get_column(c,data,col_name):
  
    if len(c) == 2:
        column = data.field(c[0]) - data.field(c[1])
    else:
        column = data.field(c[0])
        if len(c) > 2:
            print(col_name + "has >2 fields. Only the first field was used.")
  
    return column

def load(cx,p_th=0.5,N_th=5,mass_lim=False):
    
    raw_names = ['a31_1','a32_2','a33_3','a34_4','a36_more_than_4','a37_cant_tell']
    raw_cols = ['t11_arms_number_' + name + '_weighted_fraction' for name in raw_names]

    gal_data = fits.getdata(source_dir + data_file,1) 
    debiased_values = fits.getdata(source_dir + debiased_file,1)
            
    x_column = get_column(c=cx,data=gal_data,col_name="x") 
    
    f_v = np.array([debiased_values.field(c) 
                    for c in debiased_values.columns.names]).T
    f_raw = np.array([gal_data.field(c) for c in raw_cols]).T
    table = np.concatenate([f_v,f_raw,np.array([x_column]).T],axis=1)
    table = Table(table,names=('p_1','p_2','p_3','p_4','p_5','p_ct'
                               ,'p_1r','p_2r','p_3r','p_4r','p_5r','p_ctr'
                               ,'x'))
        
    p_spiral = (
        gal_data.field("t01_smooth_or_features_a02_features_or_disk_debiased")
        *gal_data.field("t02_edgeon_a05_no_debiased")
        *gal_data.field("t04_spiral_a08_spiral_debiased"))
    N_spiral = (gal_data.field("t04_spiral_a08_spiral_count")) # Load values to
    # allow data cuts to be made.
    
    spiral_select = (p_spiral > p_th) & (N_spiral >= N_th)
    finite_select = (np.isfinite(table['x']))
    
    if mass_lim == True:
        mass_cut = gal_data.field('LOGMSTAR_BALDRY06') > 10.6
        full_table = table[finite_select & mass_cut]
        spiral_table = table[finite_select & spiral_select & mass_cut]
        
    else:
        full_table = table[finite_select]
        spiral_table = table[finite_select & spiral_select]
        
    return full_table,spiral_table

In [3]:
def assign(table,debiased_table,th=0,redistribute=False,rd_th=0,ct_th=0,print_sizes=False,raw=False):
    
    m_columns = debiased_table.colnames
    
    if raw == True:
        m_array = np.array([table[column] for column in m_columns]).T
    else:
        m_array = np.array([debiased_table[column] for column in m_columns]).T
    
    arm_assignments = np.ones(len(table))*(-999) # Assigned arm numbers 
    # initially is an array of -999s. -999 means 'no assignment'.
    for m in range(6):
        a = (np.argmax(m_array,axis=1) == m) & (m_array[:,m] >= th)
        arm_assignments[a] = m
        
    if redistribute is True: # Redistribute according to thresholds.
        for m in range(5):
            arm_assignments[(np.argmax(m_array[:,:5],axis=1) == m) 
                & (arm_assignments == 5) & (table[:,m]/table[:,5] > rd_th) 
                & (table[:,5] <= ct_th)] = m

    if print_sizes is True:
        print("total sample: " + str(len(arm_assignments)))
        print("total 'assigned' sample: " 
              + str(np.sum(arm_assignments != -999)))
        for m in range(6):
            print("m = " + str(m+1) + ": " 
                  + str(np.sum(arm_assignments == m)))
      
    return arm_assignments

In [4]:
# Load the tables

data = fits.getdata(source_dir + data_file,1)
data_table = Table(data)
debiased = fits.getdata(source_dir + debiased_file,1)
debiased_table = Table(debiased)

In [5]:
p_th = 0.5
N_th = 5

p_spiral = (data.field("t01_smooth_or_features_a02_features_or_disk_debiased")
            *data.field("t02_edgeon_a05_no_debiased")
            *data.field("t04_spiral_a08_spiral_debiased"))
N_spiral = (data.field("t04_spiral_a08_spiral_count")) # Load values to
# allow data cuts to be made.
    
select_spiral = (p_spiral > p_th) & (N_spiral >= N_th)
spirals = data_table[select_spiral]
deb_spirals = debiased_table[select_spiral]

In [13]:
# Debiased vs raw comparison table?

def get_debiasing_table(table,deb):
    
    values = np.zeros((12,4))

    for mass_lim in [False,True]:
        
        for raw in [True,False]:
        
            if mass_lim == True:
                mass_select = table['LOGMSTAR_BALDRY06'] >= 10.6
                t = table[mass_select]
                d = deb[mass_select]
                r = 6
            else:
                t = table
                d = deb
                r = 0
                
            if raw is True:
                c = 0
            else:
                c = 2
                
            m_vals = assign(t,d,raw=raw)
            
            N = len(t)
            f = 1.00
            
            values[r,c:c+2] = np.array([N,f])
            r = r + 1
            
            for m in range(5):
                
                N_m = np.sum(m_vals == m)
                f = N_m/N
                
                values[r,c:c+2] = np.array([N_m,f])
                r = r + 1

    col1 = ['Volume-limited','$m=1$','$m=2$','$m=3$','$m=4$','$m=5+$',
            'Stellar mass-limited','$m=1$','$m=2$','$m=3$','$m=4$','$m=5+$']

    values[:,1] = np.round(values[:,1],decimals=2)
    values[:,3] = np.round(values[:,3],decimals=2)

    table_out = np.concatenate([np.array([col1]).T,values],axis=1)
    table_out = tabulate(table_out,headers = [r'Sample',r'$N_{raw}$',r'$f_{raw}$',r'$N_{debiased}$',r'$f_{debiased}$'],
                     tablefmt='latex')
    
    print(table_out)
    
    return None

In [89]:
def get_main_table(spirals,deb,mass_lim=True,
                   fields=[['LOGMSTAR_BALDRY06'],['PETROMAG_MG','PETROMAG_MR'],['ssfr_total_avg']]):
    
    row = 0
    
    if mass_lim is True:
        select_mass = spirals['LOGMSTAR_BALDRY06'] >= 10.6
        spirals = spirals[select_mass]
        deb = deb[select_mass]
    
    values = np.zeros((6,len(fields)+2))
    m_vals = assign(spirals,deb)
    
    N = len(spirals)
    f = 1.00
    
    means = get_property_means(spirals,fields)
    values[row]  = [N] + [f] + means.tolist()
    row = row + 1
    
    for m in range(5):
        
        N_m = np.sum(m_vals == m)
        f = N_m/N
        
        spirals_m =spirals[m_vals == m]
        means = get_property_means(spirals_m,fields)
        values[row] = [N_m] + [f] + means.tolist()
        row = row + 1
        
    col1 = ['Stellar mass-limited','$m=1$','$m=2$','$m=3$','$m=4$','$m=5+$']
    
    values[:,1:] = np.round(values[:,1:],decimals=2)

    table_out = np.concatenate([np.array([col1]).T,values],axis=1)
    
    table_out = tabulate(table_out,headers = [r'Sample',r'$N$',r'$f$',r'Stellar mass',r'$g-r$',
                                              r'$\logsSFR$'],tablefmt='latex')
    
    print(table_out)
    
    return None

In [79]:
def get_property_means(data,fields):
    
    means = np.zeros(len(fields))
    
    for c in enumerate(fields):
        if len(c[1]) == 2:
            column = data.field(c[1][0]) - data.field(c[1][1])
        else:
            column = data.field(c[1][0])
        
        column = column[np.isfinite(column)]
        means[c[0]] = np.mean(column)
        
    return means

In [91]:
get_main_table(spirals,deb_spirals,mass_lim=False)

\begin{tabular}{lrrrrr}
\hline
 Sample               &   \$N\$ &   \$f\$ &   Stellar mass &   \$g-r\$ &   \$\textbackslash{}logsSFR\$ \\
\hline
 Stellar mass-limited & 14992 &  1    &          10.61 &    0.56 &       -10.48 \\
 \$m=1\$                &   321 &  0.02 &          10.58 &    0.57 &       -10.39 \\
 \$m=2\$                &  7906 &  0.53 &          10.61 &    0.59 &       -10.5  \\
 \$m=3\$                &  2023 &  0.13 &          10.61 &    0.53 &       -10.38 \\
 \$m=4\$                &   795 &  0.05 &          10.63 &    0.52 &       -10.43 \\
 \$m=5+\$               &   744 &  0.05 &          10.67 &    0.53 &       -10.58 \\
\hline
\end{tabular}
